In [1]:
import duckdb
import pandas as pd
import json
import os

# Path to your JSON file
json_path = '../../public/data/repo_metadata.json'  # Replace with your actual file path
db_path = 'github_metadata.duckdb'      # Output DB file

# Load the JSON data
with open(json_path, 'r') as f:
    data = json.load(f)

# Ensure data is a list of repos
if isinstance(data, dict):
    data = [data]

# Flatten top-level fields
repos = []
languages = []
topics = []

for repo in data:
    repo_id = repo.get("nameWithOwner")

    # Flatten main repo info
    repos.append({
        "nameWithOwner": repo_id,
        "owner": repo.get("owner"),
        "name": repo.get("name"),
        "stars": repo.get("stars"),
        "forks": repo.get("forks"),
        "watchers": repo.get("watchers"),
        "isFork": repo.get("isFork"),
        "isArchived": repo.get("isArchived"),
        "languageCount": repo.get("languageCount"),
        "topicCount": repo.get("topicCount"),
        "diskUsageKb": repo.get("diskUsageKb"),
        "pullRequests": repo.get("pullRequests"),
        "issues": repo.get("issues"),
        "description": repo.get("description"),
        "primaryLanguage": repo.get("primaryLanguage"),
        "createdAt": repo.get("createdAt"),
        "pushedAt": repo.get("pushedAt"),
        "defaultBranchCommitCount": repo.get("defaultBranchCommitCount"),
        "license": repo.get("license"),
        "assignableUserCount": repo.get("assignableUserCount"),
        "codeOfConduct": repo.get("codeOfConduct"),
        "forkingAllowed": repo.get("forkingAllowed"),
        "parent": repo.get("parent")
    })

    for lang in repo.get("languages", []):
        languages.append({
            "repo": repo_id,
            "language": lang.get("name"),
            "size": lang.get("size")
        })

    for topic in repo.get("topics", []):
        topics.append({
            "repo": repo_id,
            "topic": topic.get("name"),
            "stars": topic.get("stars")
        })

# Convert to DataFrames
df_repos = pd.DataFrame(repos)
df_languages = pd.DataFrame(languages)
df_topics = pd.DataFrame(topics)

# Create DuckDB and insert tables
con = duckdb.connect(db_path)
con.execute("SET threads TO 4;")

con.register('df_repos', df_repos)
con.execute("CREATE OR REPLACE TABLE repos AS SELECT * FROM df_repos;")

con.register('df_languages', df_languages)
con.execute("CREATE OR REPLACE TABLE repo_languages AS SELECT * FROM df_languages;")

con.register('df_topics', df_topics)
con.execute("CREATE OR REPLACE TABLE repo_topics AS SELECT * FROM df_topics;")

con.close()

print(f"DuckDB database created at {db_path}")


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

DuckDB database created at repos.duckdb
